In [1]:
import os
import pandas as pd
import torch
import torchaudio
from TTS.api import TTS
from time import time
from text_split_and import split_text_into_chunks;


%run text_split_and.py

Chunk: Your input text here This is an example text to split How does it handle, longer sentences, you may ask? Let's find out!
Length: 120

Chunk: Your input text here This is an example text to split How does it handle, longer sentences, you may ask? Let's find out!
Length: 120



In [2]:
def load_replacements(file_path):
    """Load text replacements from an Excel file."""
    df = pd.read_excel(file_path)
    # df = df[df['langua'] != 'es']
    replacements = {}
    for index, row in df.iterrows():
        replacements[row['original']] = row['replace']
    return replacements

def replace_text(text, replacements):
    """Replace text based on a dictionary of replacements."""
    for original, replace in replacements.items():
        text = text.replace(original, replace)
    return text

In [3]:
# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [4]:
# Load replacements from Excel file
excel_file = 'replaces_words.xlsx'
replacements = load_replacements(excel_file)

In [5]:

# Ruta donde se deben eliminar las carpetas
output_path = 'output_es_short'

# Eliminar todas las carpetas en output_path
if os.path.exists(output_path):
    # Iterar sobre los elementos dentro del directorio
    for item in os.listdir(output_path):
        item_path = os.path.join(output_path, item)
        # Eliminar archivos
        if os.path.isfile(item_path):
            os.remove(item_path)
        # Eliminar carpetas y su contenido de forma recursiva
        elif os.path.isdir(item_path):
            for root, dirs, files in os.walk(item_path, topdown=False):
                for name in files:
                    os.remove(os.path.join(root, name))
                for name in dirs:
                    os.rmdir(os.path.join(root, name))
            os.rmdir(item_path)  # Finalmente eliminar la carpeta vacía
else:
    print(f"The folder {output_path} does not exist.")

In [6]:
# Asegurarse de que la carpeta exista después de eliminar su contenido
os.makedirs(output_path, exist_ok=True)

# Ruta de la carpeta de entrada con los archivos .txt
folder_path = '/mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/0_create text/text/es/shorts/'

# Lista de archivos .txt en la carpeta
files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

# Ordenar los archivos si es necesario
files.sort()

In [7]:
Path_voice = '/mnt/D8E84E4DE84E2A58/Env_python/Machine_learing_Test/0_Create_audio/data/wavs/'

In [8]:
# Init TTS

tts = TTS("xtts").to(device)
# tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

for i, file in enumerate(files):
    file_path = os.path.join(folder_path, file)
    new_folder_path = os.path.join(output_path, f'{i+1}')
    os.makedirs(new_folder_path, exist_ok=True)
    print(f"Procesando: {file} en {new_folder_path}")

    # Leer el contenido del archivo .txt
    with open(file_path, 'r', encoding='utf-8') as f:
        text_content = f.read()

    text = replace_text(text_content, replacements)

    # Dividir el texto en fragmentos manejables
    separated_input = split_text_into_chunks(text)

    # Inicializar lista de clips de audio
    all_audio_parts = []

    # Generar audios por fragmento
    for index, text in enumerate(separated_input):
        audio_file_path = os.path.join(new_folder_path, f"{index}.wav")
        wav_data = tts.tts_to_file(
            text=text,
            speaker_wav=[f"{Path_voice}complete/sample_1.wav", f"{Path_voice}sample0001.wav", f"{Path_voice}sample0002.wav", f"{Path_voice}sample0003.wav", f"{Path_voice}sample0004.wav", f"{Path_voice}sample0005.wav", f"{Path_voice}sample0006.wav", f"{Path_voice}sample0007.wav", f"{Path_voice}sample0008.wav"],
            language="es",
            temperature=0.9,
            file_path=audio_file_path
        )
        print(f"Audio generado: {audio_file_path}")
        torch.cuda.empty_cache()  # Liberar memoria GPU
        audio_part, _ = torchaudio.load(audio_file_path)
        all_audio_parts.append(audio_part)

    # Concatenar todos los clips de audio
    concatenated_audio = torch.cat(all_audio_parts, dim=-1)

    # Guardar el audio concatenado
    final_audio_path = os.path.join(new_folder_path, f"audio_final_{i+1}.wav")
    torchaudio.save(final_audio_path, concatenated_audio, sample_rate=24000)
    print(f"Audio final guardado en: {final_audio_path}")


print(f"Se procesaron {len(files)} archivos y se generaron carpetas correspondientes en {output_path}.")

/home/andresmtr/miniconda3/envs/AndTTSCoqui/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 > Using model: xtts


/home/andresmtr/miniconda3/envs/AndTTSCoqui/lib/python3.9/site-packages/TTS/tts/layers/xtts/xtts_manager.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.speakers = 

Procesando: 1.txt en output_es_short/1
Chunk: Spotify Wrapped 2024 añade un podcast impulsado por Inteligencia Artificial de la mano de NotebookLM de Google
Length: 110

Chunk: OpenAI recibe una nueva inversión de 1,5 mil millones de dólares de SoftBank,
Length: 77

Chunk: permitiendo a los empleados vender acciones en una oferta pública
Length: 65

Chunk: La PS5 incluirá temas clásicos y secuencias de inicio para celebrar el 30º aniversario de PlayStation
Length: 101

 > Text splitted to sentences.
['Spotify Wrapped 2024 añade un podcast impulsado por Inteligencia Artificial de la mano de NotebookLM de Google']


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


 > Processing time: 2.6548848152160645
 > Real-time factor: 0.3184579281025014
Audio generado: output_es_short/1/0.wav
 > Text splitted to sentences.
['OpenAI recibe una nueva inversión de 1,5 mil millones de dólares de SoftBank,']
 > Processing time: 1.5262174606323242
 > Real-time factor: 0.21583565294345017
Audio generado: output_es_short/1/1.wav
 > Text splitted to sentences.
['permitiendo a los empleados vender acciones en una oferta pública']
 > Processing time: 1.1021373271942139
 > Real-time factor: 0.25378162139340454
Audio generado: output_es_short/1/2.wav
 > Text splitted to sentences.
['La PS5 incluirá temas clásicos y secuencias de inicio para celebrar el 30º aniversario de PlayStation']
 > Processing time: 1.685783863067627
 > Real-time factor: 0.2125739670866569
Audio generado: output_es_short/1/3.wav
Audio final guardado en: output_es_short/1/audio_final_1.wav
Procesando: 2.txt en output_es_short/2
Chunk: Spotify ha lanzado su función anual Wrapped para 2024
Length: 53
